In [16]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [20]:
import os
import pandas as pd

MANDATORY_METRICS = {
    "Impressions": "impressions",
    "Members reached": "members_reached",
    "Profile viewers from this post": "profile_viewers",
    "Followers gained from this post": "followers_gained",
    "Reactions": "reactions",
    "Comments": "comments",
    "Reposts": "reposts",
    "Saves": "saves",
    "Sends on LinkedIn": "sends",
}

IST_TZ = "Asia/Kolkata"

def clean_numeric(value):
    if pd.isna(value):
        return None
    if isinstance(value, str):
        value = value.replace(",", "").strip()
    return pd.to_numeric(value, errors="coerce")

def load_linkedin_post_analytics(directory_path: str) -> pd.DataFrame:
    posts = []

    for file in os.listdir(directory_path):
        if not file.lower().endswith(".xlsx"):
            continue

        file_path = os.path.join(directory_path, file)
        df_raw = pd.read_excel(file_path, header=0)

        post_url = df_raw.columns[1]
        df_raw.columns = ["metric", "value"]
        df_raw = df_raw.dropna(subset=["metric"])

        metric_map = dict(zip(df_raw["metric"], df_raw["value"]))

        post_date = metric_map.get("Post Date")
        post_time = metric_map.get("Post Publish Time")

        # UTC → IST datetime
        post_datetime_utc = pd.to_datetime(
            f"{post_date} {post_time}",
            errors="coerce",
            utc=True
        )
        post_datetime_ist = post_datetime_utc.tz_convert(IST_TZ)

        post_data = {
            "post_url": post_url,
            "post_date": post_datetime_ist.date() if pd.notna(post_datetime_ist) else None,
            "post_time": post_datetime_ist.time() if pd.notna(post_datetime_ist) else None,
            "post_datetime": post_datetime_ist,
        }

        for raw_key, clean_key in MANDATORY_METRICS.items():
            post_data[clean_key] = clean_numeric(metric_map.get(raw_key))

        posts.append(post_data)

    return pd.DataFrame(posts)


In [21]:
df = load_linkedin_post_analytics(r'''/Users/etqadkhan/Downloads/linkedin_post_impressions''')

In [22]:
df = df.sort_values(["post_datetime"]).reset_index(drop=True)

In [23]:
df

,post_url,post_date,post_time,post_datetime,impressions,members_reached,profile_viewers,followers_gained,reactions,comments,reposts,saves,sends
0,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-02,18:02:00,2026-01-02 18:02:00+05:30,1047,628,8,0,21,0,0,2,0
1,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-03,12:16:00,2026-01-03 12:16:00+05:30,863,562,7,2,9,2,0,3,0
2,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-04,12:58:00,2026-01-04 12:58:00+05:30,1003,657,5,2,11,3,0,4,0
3,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-05,18:00:00,2026-01-05 18:00:00+05:30,881,572,7,0,24,0,0,3,0
4,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-06,18:00:00,2026-01-06 18:00:00+05:30,681,457,3,1,6,1,0,1,0
5,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-07,11:01:00,2026-01-07 11:01:00+05:30,918,629,6,0,12,0,0,5,0
6,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-08,09:12:00,2026-01-08 09:12:00+05:30,517,336,6,2,11,2,0,1,0
7,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-09,16:00:00,2026-01-09 16:00:00+05:30,412,274,3,0,4,0,0,0,0
8,https://www.linkedin.com/feed/update/urn:li:ug...,2026-01-10,15:36:00,2026-01-10 15:36:00+05:30,597,388,1,0,12,0,0,2,0
9,https://www.linkedin.com/feed/update/urn:li:sh...,2026-01-11,11:58:00,2026-01-11 11:58:00+05:30,504,301,5,0,15,1,0,0,0
